**Assignment 1**
----------------------------------
**Name** : *Ahmad Farhan* <br>
**Roll No.** : *i211366* <br>
**Section** : *A* <br>
__________________________________

In [71]:
import heapq                    # UCS
from collections import deque   # BFS, DFS

In [72]:
class Graph:
    '''
    Adjecency List based Graph Data Structure
    Attributes:
        graph (dict): Dictionary mapping vertices to (parent_set, cost)
        nkeys (int): Total number of vertices in graph
    '''
    def __init__(self):
        '''Constructor: Initializes graph attributes'''
        self.graph = dict()
        self.nkeys = 0

    def add_edge(self, vertex, parent_set, cost):
        ''' Adds a potential edge as vertex: (parent_set, cost)'''
        if vertex not in self.graph: 
            self.graph[vertex] = []; self.nkeys += 1
        self.graph[vertex].append((parent_set, cost))

    def get_vertices(self):
        ''' Returns a tuple of all the vertices in graph '''
        return tuple(self.graph.keys())

    def mcc(self, vertex, ordering):
        ''' 
        Minimum Cost Computation for a given vertex and ordering
        Args: 
            Vertex (int): Vertex for which mininum cost is being calculated
            ordering (tuple): Current Ordering of Vertices
        Returns: (float) Minimum Cost of vertex within given ordering
        '''
        min_cost = float('inf')
        # Initializing a boolean array as a hashtable
        ordering_hash = [False] * (self.nkeys + 1)
        # Mark Vertices Available in ordering as True
        for v in ordering: ordering_hash[v] = True

        # Iterate over each parent_set, cost pair associated with current vertex
        for parent_set, cost in self.graph[vertex]:
            # If all vertices of parent_set are available in ordering
            if all(ordering_hash[parent] for parent in parent_set):
                min_cost = min(min_cost, cost)  # Update minimum cost
        
        return min_cost

    def new_node(self, vertex, total_cost, ordering, remaining):
        '''
        Create a Node to explore later
        Args:
            vertex (int): The vertex to search.
            total_cost (float): The total cost so far.
            ordering (tuple): The current ordering of vertices.
            remaining (tuple): The remaining vertices to explore.
        Returns: (tuple) updated cost, ordering, and remaining vertices
        '''
        # Add current vertex to ordering
        new_ordering = ordering + (vertex,)
        # Remove current vertex from remaining vertices to explore
        new_remaining = tuple(v for v in remaining if v != vertex)
        # Add cost till now with minimum possible cost within ordering
        new_total_cost = total_cost + self.mcc(vertex, ordering)
        return new_total_cost, new_ordering, new_remaining

In [73]:
def parse_file(filename):
    ''' Parses data from file to construct a graph
        Args: Filename (str)
        Returns: Graph (graph)
    '''
    file = open(f'../Assignments/Data/{filename}')
    graph = Graph() 

    for line in file:
        # First and Last values are vertex and parent set edge cost (ignore the rest)
        vertex, *dump, cost = line.split(',')

        # Isolate Parent Set and remove braces
        parents = line.split('{')[1].split('}')[0]

        # Convert parent of the set to vertices into a tuple
        pset = tuple(int(p) for p in parents.split(',')) if parents != '' else tuple()
        graph.add_edge(int(vertex), pset, float(cost))
    
    file.close()
    return graph

In [74]:
# Uniform Cost Search
def ucs(graph, vertices):
        
    # Initialize open queue with initial state
    frontier = []
    heapq.heappush(frontier, (0, tuple(), vertices))

    while(frontier):
        # Pop Node with lowest cost
        total_cost, ordering, remaining = heapq.heappop(frontier)

        # If all vertices have been visited
        if len(ordering) == len(vertices): break
        else: # Create all possible child orderings from current ordering
            for vertex in remaining:
                node = graph.new_node(vertex, total_cost, ordering, remaining)
                heapq.heappush(frontier, node)  # Add searchable ordering node to heap

    return total_cost, ordering

In [75]:
# Breadth First Search
def bfs(graph, vertices):

    # Initialize open queue with initial state
    frontier, min_cost = deque(), float('inf')
    frontier.append((0, tuple(), vertices))
    min_ordering = None

    while(frontier):
        # Pop Node as FIFO (queue)
        total_cost, ordering, remaining = frontier.popleft()

        # If no remaining, then possible min_ordering obtained
        if not remaining:
            if total_cost < min_cost:
                min_cost = total_cost
                min_ordering = ordering
        else: # Create all possible child orderings from current ordering
            for vertex in remaining:
                node = graph.new_node(vertex, total_cost, ordering, remaining)
                frontier.append(node)   # Add searchable ordering node to queue

    return min_cost, min_ordering

In [76]:
# Deapth First Search 
def dfs(graph, vertices):
    
    # Initialize open queue with initial state
    frontier, min_cost = deque(), float('inf')
    frontier.append((0, tuple(), vertices))
    min_ordering = None

    while(frontier):
        # Pop Node as FILO (stack)
        total_cost, ordering, remaining = frontier.pop()

        # If no remaining, then possible min_ordering obtained
        if not remaining:
            if total_cost < min_cost:
                min_cost, min_ordering = total_cost, ordering
        else: # Create all possible child orderings from current ordering
            for vertex in remaining:
                node = graph.new_node(vertex, total_cost, ordering, remaining)
                frontier.append(node)   # Add searchable ordering node to queue

    return min_cost, min_ordering

### Algorithm Comparisons ###

In [77]:
graph = parse_file('data0.txt')
vertices = graph.get_vertices()
print("Graph Vertices :",vertices)

Graph Vertices : (1, 2, 3, 4, 5)


In [78]:
min_cost, ordering = bfs(graph, vertices)
print(f"BFS: {min_cost:.3f} - {ordering} ")

BFS: 465.434 - (4, 2, 5, 3, 1) 


In [79]:
min_cost, ordering = dfs(graph, vertices)
print(f"DFS: {min_cost:.3f} - {ordering} ")

DFS: 465.434 - (5, 4, 3, 1, 2) 


In [80]:
min_cost, ordering = ucs(graph, vertices)
print(f"UCS: {min_cost:.3f} - {ordering} ")

UCS: 465.434 - (4, 2, 5, 3, 1) 


### Conclusion
Pure BFS, DFS and UCS are not optimal enough to run data1.txt and data3.txt <Br>
Using the known time and space complexities of these algorithms, an estimated time and space needed are as such:
<br><br>

Search Algorithm Complexities:
|               |       BFS        |       DFS        |       UFS        |
|---------------|------------------|------------------|------------------|
|Time           |   $O(b^{d+1})$   |     $O(b^m)$     |  $O(b^(C^*/e))$  |
|Space          |   $O(b^{d+1})$   |     $O(bm)$      |  $O(b^(C^*/e))$  |


***data1.txt*** <br>
- Vertices = 18
- Edges = 220
- b = Vertices = 18
- d = m = Vertices = 18
- Search Space = |Orderings| = 6.40237e+15

**data2.txt** <br>
- Vertices = 19
- Edges = 720
- b = Vertices
- d = m = Vertices
- Search Space = |Orderings| = 1.21645e+17

Note: For a perspective, this laptop evaluated 7 million orderings in 1.5 hours using approx. 4 Gb RAM throughout execution. <br>
To evaluate 6.40237e+15 search space (data1.txt): <br>
Estimated time = 156,555 years
__________________________________________________________

#### Extras (Attempted Optimization)
Following algorithms are optimizations of above search algorithms where drastically fewer orderings are considered to give a close approximation of minimum (not always actual minimum). Runtime of data1.txt brought down to 52 mins <br>

Note: Consider below code only as an attempt at optimization. If only one solution is to be accepted then please consider above (slower) versions as final as they offer completeness with optimality

In [84]:
def mcc(parents, next_vertex, visited):
    min_cost = float('inf')
    for parent_set, cost in parents:
        if (all(parent not in visited for parent in parent_set)):
            if next_vertex in parent_set:
                min_cost = min(min_cost, cost)
    return min_cost

def initqueue(graph, start_vertex):
    '''Initialize Queue with possible parents of starting vertices'''
    queue = deque() 
    parent_sets = graph.graph[start_vertex]

    for next_vertex, cost in parent_sets:
        if(len(next_vertex) == 1):
            visited = next_vertex + (start_vertex,)
            c = []
            queue.append((next_vertex[0], visited, mcc(parent_sets, next_vertex[0], c)))
    return queue

def initheap(graph, start_vertex):
    '''Initialize Heap with possible parents of starting vertices'''
    queue = [] 
    parent_sets = graph.graph[start_vertex]

    for next_vertex, cost in parent_sets:
        if(len(next_vertex) == 1):
            visited = next_vertex + (start_vertex,)
            heapq.heappush(queue, (cost, next_vertex[0], visited))
    return queue

def mccv(tordering, graph):
    '''Calculates min cost of given ordering'''
    visited, total_cost = [], 0
    for vertex in tordering:
        min_cost = float('inf')
        nodes = graph[vertex]
        for parent_set in nodes:
            if all(parent in visited for parent in parent_set):
                min_cost = min(min_cost, nodes[parent_set])
        total_cost += min_cost
        visited.append(vertex)
    return total_cost

def obfs(graph, vertex): pass
def odfs(graph, vertex): pass
def oucs(graph, vertex): pass

def run_algo(algo):
    '''Control Function of optimized versions of algorithms'''
    algorithm = None
    if algo == 'bfs': algorithm = obfs
    elif algo == 'dfs': algorithm = odfs
    elif algo == 'ucs': algorithm = oucs
    else: print("Invalid Algorithm"); exit()

    s = []
    for vertex in vertices: s += (algorithm(graph, vertex))
    for ordering, cost in s: cost = mccv(ordering,graph.graph)
    s = sorted(s, key=lambda x:x[1])
    print("\nFinal Results :", s[0])
    print("Orderings considered: ", len(s))


In [82]:
def obfs(graph, start_vertex):
    '''Optimzed BFS'''
    orderings = []
    queue = initqueue(graph, start_vertex)

    while queue:
        vertex, visited, total_cost = queue.popleft()
        parent_sets = graph.graph[vertex]

        if len(visited) < graph.nkeys:
            for next_vertex,_ in parent_sets:
                if(len(next_vertex) == 1 and next_vertex[0] not in visited):
                    new_cost = total_cost + mcc(parent_sets, next_vertex[0], visited)
                    new_visited = next_vertex + visited
                    queue.append((next_vertex[0], new_visited, new_cost))
        else:
            emptyset, cost = graph.graph[vertex][0]
            total_cost += cost
            orderings.append((visited, total_cost))
    return orderings

def odfs(graph, start_vertex):
    '''Optimzed DFS'''
    orderings = []
    queue = initqueue(graph, start_vertex)

    while queue:
        vertex, visited, total_cost = queue.pop()
        parent_sets = graph.graph[vertex]

        if len(visited) < graph.nkeys:
            for next_vertex,_ in parent_sets:
                if(len(next_vertex) == 1 and next_vertex[0] not in visited):
                    new_cost = total_cost + mcc(parent_sets, next_vertex[0], visited)
                    new_visited = next_vertex + visited
                    queue.append((next_vertex[0], new_visited, new_cost))
        else:
            emptyset, cost = graph.graph[vertex][0]
            total_cost += cost
            orderings.append((visited, total_cost))
    return orderings

def oucs(graph, start_vertex):
    '''Optimzed UCS'''
    orderings = []
    queue = initheap(graph, start_vertex)

    while queue:
        total_cost, vertex, visited = heapq.heappop(queue)
        parent_sets = graph.graph[vertex]

        if len(visited) < graph.nkeys:
            for next_vertex,_ in parent_sets:
                if(len(next_vertex) == 1 and next_vertex[0] not in visited):
                    new_cost = total_cost + mcc(parent_sets, next_vertex[0], visited)
                    new_visited = next_vertex + visited
                    heapq.heappush(queue, (new_cost, next_vertex[0], new_visited))  # Add searchable ordering node to heap
        else:
            emptyset, cost = graph.graph[vertex][0]
            total_cost += cost
            orderings.append((visited, total_cost))
    return orderings

In [ ]:
algorithms = ['bfs', 'dfs', 'ucs']
for algorithm in algorithms:
    run_algo(algorithm)